<a href="https://colab.research.google.com/github/JayThibs/ai-safety-scrape/blob/main/scrape_ai_alignment_content.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting Contents from AI Alignment Resources


# Installations

In [1]:
!pip install tika

  Created wheel for tika: filename=tika-1.24-py3-none-any.whl size=32893 sha256=0924a8014449a7c27ca4a805366e0a3914631a2597ba01840529132cb996ba68
  Stored in directory: /root/.cache/pip/wheels/ec/2b/38/58ff05467a742e32f67f5d0de048fa046e764e2fbb25ac93f3
Successfully built tika


# Imports

In [4]:
from tika import parser
from google.colab import drive

# Setting up Environment

In [6]:
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
%cd drive/MyDrive/data/ai-alignment-dataset

/content/drive/MyDrive/data/ai-alignment-dataset


# Extract text from AI Alignment Resources

In [10]:
def tikaTextExtractor(file_path):
    """Extracts text from a PDF using tika."""
    print("Extracting text from file: " + file_path)
    parsed_tika = parser.from_file(file_path)
    return parsed_tika["content"][0:1000]

In [12]:
tikaTextExtractor('2112.00861.pdf')

Extracting text from file: 2112.00861.pdf


'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nA General Language Assistant\nas a Laboratory for Alignment\n\nAmanda Askell∗ Yuntao Bai∗ Anna Chen∗ Dawn Drain∗ Deep Ganguli∗ Tom Henighan†\n\nAndy Jones† Nicholas Joseph† Ben Mann∗ Nova DasSarma Nelson Elhage\n\nZac Hatfield-Dodds Danny Hernandez Jackson Kernion Kamal Ndousse\n\nCatherine Olsson Dario Amodei Tom Brown Jack Clark Sam McCandlish Chris Olah\n\nJared Kaplan‡\n\nAnthropic\n\nAbstract\n\nGiven the broad capabilities of large language models, it should be possible to work towards\na general-purpose, text-based assistant that is aligned with human values, meaning that it is\nhelpful, honest, and harmless. As an initial foray in this direction we study simple baseline\ntechniques and evaluations, such as prompting. We find that the benefits from modest\ninterventions increase with model size, generalize to a variety of alignment evaluations, and\ndo not comprom